# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [14]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [15]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets                    # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                 # Evaluación de sobremuestreo con partición hold-out
from custom_samplers import PCSMOTEWrapper                     # Wrapper personalizado para la técnica PCSMOTE
from pc_smote import PCSMOTE                                   # Implementación principal de PCSMOTE
# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import gc, os, time                                                      # Operaciones con el sistema de archivos

from sklearn.experimental import enable_halving_search_cv  # noqa: F401  <- necesario
from sklearn.model_selection import HalvingRandomSearchCV

# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from sklearn.pipeline import make_pipeline, Pipeline            # Creación de pipelines de procesamiento y modelado

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)
from scipy.stats import loguniform, randint, uniform

from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Evitar sobre-suscripción de CPU (BLAS/OpenMP)
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")
SCORING_PRIMARY = 'f1_macro'  # único métrico para Halving

### Evaluación de modelos con validación cruzada estratificada

Para evaluar el rendimiento de los modelos de clasificación sobre los datasets previamente balanceados, se utilizó validación cruzada estratificada de 5 particiones (Stratified K-Fold con *k=5*). Este método garantiza que en cada fold de entrenamiento y validación se preserve la proporción original de clases, lo cual es especialmente importante en tareas de clasificación multiclase con datasets balanceados artificialmente.

Durante el proceso, cada modelo es entrenado y evaluado cinco veces, cada vez usando un subconjunto distinto como conjunto de prueba y el resto como conjunto de entrenamiento. Las métricas calculadas en cada iteración (F1-score macro, balanced accuracy, MCC y kappa de Cohen) se promedian para obtener un valor representativo y del rendimiento general del modelo sobre ese dataset aumentado.

Este enfoque evita sobreajuste y proporciona una evaluación más confiable que una simple división train/test, permitiendo comparar de forma justa distintas configuraciones de sobremuestreo y modelos de clasificación.


In [16]:
import os, gc, time
from dataclasses import dataclass, asdict
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    f1_score,
    balanced_accuracy_score,
    matthews_corrcoef,
    cohen_kappa_score,
    accuracy_score,
    recall_score,
    roc_auc_score,
    average_precision_score,
    make_scorer,
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from scipy.stats import randint, uniform, loguniform

# =========================
# Configuración general
# =========================
RUTA_DATASETS_BASE = "../datasets/datasets_aumentados/base/"
RUTA_DATASETS_AUMENTADOS = "../datasets/datasets_aumentados/"
DIRECTORIO_SALIDA = "../resultados"
os.makedirs(DIRECTORIO_SALIDA, exist_ok=True)
RUTA_DATASETS_CLASICOS = "../datasets/datasets_aumentados/resampler_clasicos/"
os.makedirs(RUTA_DATASETS_CLASICOS, exist_ok=True)

RANDOM_STATE = 42
SCORING_REFIT = "f1_macro"
SCORING_MULTIPLE = {
    "f1_macro": "f1_macro",
    "balanced_accuracy": "balanced_accuracy",
    "mcc": make_scorer(matthews_corrcoef),
    "cohen_kappa": make_scorer(cohen_kappa_score),
}
N_ITER_BUSQUEDA_POR_DEFECTO = 40
OMITIR_SVM_EN_SHUTTLE_AUMENTADO = True  # política opcional
NOMBRE_ARCHIVO_EXCEL = os.path.join(DIRECTORIO_SALIDA, "resultados_RS_cv_vs_test.xlsx")

# =========================
# Estructuras de datos
# =========================
@dataclass
class DatasetCombination:
    """Representa una combinación base o aumentada (metadatos y paths)."""
    dataset_logico: str          # p. ej. "us_crime"
    tipo_combination: str        # "base" | "aumentado"
    ruta_train_csv: str
    ruta_test_csv: str
    tecnica_aumento: str = "base"
    valor_densidad: str = "--"
    valor_riesgo: str = "--"
    criterio_pureza: str = "--"
    grado_limpieza: str = "--"   # nuevo: I0, I1, I5, etc.

@dataclass
class RegistroRendimiento:
    dataset_logico: str
    tipo_combination: str
    nombre_modelo_aprendizaje: str
    tecnica_aumento: str
    valor_densidad: str
    valor_riesgo: str
    criterio_pureza: str
    grado_limpieza: str
    cantidad_train: int
    cantidad_test: int
    cantidad_caracteristicas: int
    # CV (sobre el mejor candidato de RS)
    cv_f1_macro: float
    cv_balanced_accuracy: float
    cv_mcc: float
    cv_cohen_kappa: float
    # Test (macro y adicionales)
    test_f1_macro: float
    test_balanced_accuracy: float
    test_mcc: float
    test_cohen_kappa: float
    test_accuracy: float
    test_f1_por_clase: str
    test_recall_minoritario: float
    test_gmean: float
    test_roc_auc_macro_ovr: float
    test_auc_pr_macro: float
    # Auditoría
    mejores_hiperparametros: str
    tiempo_busqueda_seg: float



# =========================
# Utilidades de datos
# =========================
def cargar_matriz_caracteristicas_y_etiquetas_desde_csv(ruta_csv):
    """Lee un CSV y devuelve (X, y). Si existe 'target', la usa; si no, usa la última columna como y."""
    df = pd.read_csv(ruta_csv)
    if "target" in df.columns:
        X = df.drop(columns=["target"]).to_numpy(dtype=np.float32, copy=False)
        y = df["target"].to_numpy()
    else:
        X = df.iloc[:, :-1].to_numpy(dtype=np.float32, copy=False)
        y = df.iloc[:, -1].to_numpy()
    return X, y

def enumerar_combinaciones_base_y_aumentadas(
    ruta_base,
    ruta_aumentados,
    ruta_clasicos=None,
    verbose=True
):
    """
    Descubre combinaciones base y aumentadas.

    - Base:
        {dataset}_train.csv  +  {dataset}_test.csv  en ruta_base.
    - Clásicos:
        smote_{dataset}_train.csv,
        borderlinesmote_{dataset}_train.csv,
        adasyn_{dataset}_train.csv
        en ruta_clasicos, usando SIEMPRE como test: ruta_base/{dataset}_test.csv
    - Aumentados PCSMOTE:
        pcsmote_{dataset}_Dxx_Ryy_Pzzz_Ikk_train.csv en ruta_aumentados,
        usando SIEMPRE como test: ruta_base/{dataset}_test.csv

    Devuelve una lista de DatasetCombination.
    """
    combinaciones = []
    tests_base_por_dataset = {}

    # ===================== BASES =====================
    if os.path.isdir(ruta_base):
        if verbose:
            print(f"📂 Explorando carpeta base: {ruta_base}")
        for nombre in sorted(os.listdir(ruta_base)):
            if not nombre.endswith("_train.csv"):
                if verbose:
                    print(f"  ⚪ Omitido (no es *_train.csv): {nombre}")
                continue

            ruta_train = os.path.join(ruta_base, nombre)
            ruta_test = os.path.join(
                ruta_base,
                nombre.replace("_train.csv", "_test.csv")
            )

            if not os.path.isfile(ruta_test):
                if verbose:
                    print(f"  ⚠️  Falta test para {nombre}, se omite.")
                continue

            dataset_logico = nombre.replace("_train.csv", "")
            tests_base_por_dataset[dataset_logico] = ruta_test

            combinaciones.append(DatasetCombination(
                dataset_logico=dataset_logico,
                tipo_combination="base",
                ruta_train_csv=ruta_train,
                ruta_test_csv=ruta_test,
                tecnica_aumento="base",
                valor_densidad="--",
                valor_riesgo="--",
                criterio_pureza="--",
                grado_limpieza="--",
            ))
    else:
        if verbose:
            print(f"❌ No existe carpeta base: {ruta_base}")

    # ===================== CLÁSICOS (SMOTE / Borderline / ADASYN) =====================
    if ruta_clasicos is not None and os.path.isdir(ruta_clasicos):
        if verbose:
            print(f"📂 Explorando carpeta clásicos: {ruta_clasicos}")
        for nombre in sorted(os.listdir(ruta_clasicos)):
            if not nombre.endswith("_train.csv"):
                if verbose:
                    print(f"  ⚪ Omitido (no es *_train.csv): {nombre}")
                continue

            # Esperado: smote_{dataset}_train.csv, borderlinesmote_{dataset}_train.csv, adasyn_{dataset}_train.csv
            nombre_sin = nombre.replace("_train.csv", "")
            partes = nombre_sin.split("_", 1)
            if len(partes) != 2:
                if verbose:
                    print(f"  ⚠️  No cumple patrón esperado (tecnica_dataset_train.csv): {nombre}")
                continue

            tecnica = partes[0]          # smote / borderlinesmote / adasyn
            dataset_logico = partes[1]   # us_crime, glass, etc.

            ruta_train = os.path.join(ruta_clasicos, nombre)
            ruta_test = tests_base_por_dataset.get(dataset_logico)

            if ruta_test is None or not os.path.isfile(ruta_test):
                if verbose:
                    print(f"  ⚠️  No hay test base para dataset '{dataset_logico}', se omite {nombre}")
                continue

            combinaciones.append(DatasetCombination(
                dataset_logico=dataset_logico,
                tipo_combination="aumentado",   # se considera "aumentado" como PCSMOTE
                ruta_train_csv=ruta_train,
                ruta_test_csv=ruta_test,
                tecnica_aumento=tecnica,
                valor_densidad="--",
                valor_riesgo="--",
                criterio_pureza="--",
                grado_limpieza="--",
            ))
            if verbose:
                print(f"  ✅ Clásico: {nombre} -> test base: {os.path.basename(ruta_test)}")
    else:
        if verbose and ruta_clasicos is not None:
            print(f"❌ No existe carpeta clásicos: {ruta_clasicos}")

    # ===================== AUMENTADOS PCSMOTE =====================
    if os.path.isdir(ruta_aumentados):
        if verbose:
            print(f"📂 Explorando carpeta aumentados: {ruta_aumentados}")
        for nombre in sorted(os.listdir(ruta_aumentados)):
            if not nombre.endswith("_train.csv"):
                if verbose:
                    print(f"  ⚪ Omitido (no es *_train.csv): {nombre}")
                continue

            # por si alguna vez cae un base_ acá
            if nombre.startswith("base_"):
                if verbose:
                    print(f"  ⚪ Omitido (parece base): {nombre}")
                continue

            ruta_train = os.path.join(ruta_aumentados, nombre)

            # Esperado: pcsmote_{dataset}_Dxx_Ryy_Pzzz_Ikk_train.csv
            nombre_sin = nombre.replace("_train.csv", "")
            partes = nombre_sin.split("_")

            if len(partes) < 6:
                if verbose:
                    print(f"  ⚠️  No cumple patrón esperado (tecnica_dataset_Dx_Ry_Pz_Ik): {nombre}")
                continue

            tecnica = partes[0]
            dens_token = partes[-4]   # D25
            ries_token = partes[-3]   # R50
            pur_token  = partes[-2]   # PPentropia / Pproporcion
            limp_token = partes[-1]   # I0 / I1 / I5
            dataset_logico = "_".join(partes[1:-4])  # respeta nombres con '_'

            if not (
                dens_token.startswith("D")
                and ries_token.startswith("R")
                and pur_token.startswith("P")
                and limp_token.startswith("I")
            ):
                if verbose:
                    print(f"  ⚠️  Tokens D/R/P/I no válidos en: {nombre}")
                continue

            # usar SIEMPRE el test base de ese dataset
            ruta_test = tests_base_por_dataset.get(dataset_logico)
            if ruta_test is None or not os.path.isfile(ruta_test):
                if verbose:
                    print(f"  ⚠️  No hay test base para dataset '{dataset_logico}', se omite {nombre}")
                continue

            combinaciones.append(DatasetCombination(
                dataset_logico=dataset_logico,
                tipo_combination="aumentado",
                ruta_train_csv=ruta_train,
                ruta_test_csv=ruta_test,
                tecnica_aumento=tecnica,
                valor_densidad=dens_token[1:],         # "25"
                valor_riesgo=ries_token[1:],           # "50"
                criterio_pureza=pur_token[1:],         # "Pentropia"/"Pproporcion"
                grado_limpieza=limp_token[1:],         # "0"/"1"/"5"
            ))
            if verbose:
                print(f"  ✅ Aumentado: {nombre} -> test base: {os.path.basename(ruta_test)}")
    else:
        if verbose:
            print(f"❌ No existe carpeta aumentados: {ruta_aumentados}")

    if verbose:
        print(f"📊 Total combinaciones descubiertas: {len(combinaciones)}")
    return combinaciones

def definir_configuracion_busqueda_para_dataset(X_train, nombre_dataset_logico, tipo_combination):
    """
    Define la configuración de búsqueda: esquema de CV (n_splits), n_iter (RandomizedSearchCV) y n_jobs.
    Mantiene: Shuttle aumentado → 2 folds; Shuttle o n≥10k → 3 folds; resto → 5 folds.
    """
    n_muestras = X_train.shape[0]
    es_shuttle = (nombre_dataset_logico.lower() == "shuttle")
    if es_shuttle and tipo_combination == "aumentado":
        cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE)
    elif es_shuttle or n_muestras >= 10000:
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    else:
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

    cpu = os.cpu_count() or 4
    n_jobs = 1 if (es_shuttle or n_muestras >= 10000) else max(1, min(4, cpu // 2))
    n_iter = 25 if (es_shuttle or n_muestras >= 10000) else N_ITER_BUSQUEDA_POR_DEFECTO
    return dict(cv=cv, n_jobs=n_jobs, n_iter=n_iter)

# =========================
# Registro de modelos
# =========================
def construir_estimador_y_espacio_svm():
    """Devuelve (pipeline SVM, espacio de hiperparámetros) con kernels linear/rbf."""
    est = Pipeline([('classifier', SVC(random_state=RANDOM_STATE, probability=False, max_iter=5000, cache_size=400))])
    # espacio de busqueda de hiperparametros,
    # combinaciones posibles para cada
    # hiperparametro del modelo
    space = {
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__C': loguniform(1e-3, 1e2),
        'classifier__gamma': loguniform(1e-4, 1e0),  # ignorado cuando kernel='linear'
        'classifier__shrinking': [True, False],
        'classifier__class_weight': [None, 'balanced'],
    }
    return est, space

def construir_estimador_y_espacio_regresion_logistica_saga():
    """Devuelve (pipeline LR con solver saga) evitando combinaciones inválidas; penalización L1/L2."""
    est = Pipeline([('classifier', LogisticRegression(max_iter=5000, random_state=RANDOM_STATE, solver='saga'))])
    # espacio de busqueda de hiperparametros,
    # combinaciones posibles para cada
    # hiperparametro del modelo
    space = {
        'classifier__penalty': ['l1', 'l2'],
        'classifier__C': loguniform(1e-4, 1e2),
        'classifier__fit_intercept': [True, False],
        'classifier__class_weight': [None, 'balanced'],
        'classifier__tol': loguniform(1e-5, 1e-3),
        'classifier__l1_ratio': [None],  # evitamos elasticnet para simplificar
    }
    return est, space

def construir_estimador_y_espacio_random_forest():
    est = Pipeline([
        ('classifier', RandomForestClassifier(
            random_state=RANDOM_STATE,
            n_jobs=1,
            bootstrap=True,
            oob_score=False,
            n_estimators=150,
            max_depth=None,
            max_features='sqrt',
            min_samples_split=2,
            min_samples_leaf=1,
            class_weight=None,
            criterion='gini'
        ))
    ])

    # Sin espacio de búsqueda (lo podés manejar aparte)
    space = {}
    return est, space


REGISTRO_MODELOS = {
    # "LogisticRegression": construir_estimador_y_espacio_regresion_logistica_saga,
    "RandomForest": construir_estimador_y_espacio_random_forest,
    # "SVM": construir_estimador_y_espacio_svm
}
ORDEN_MODELOS = [
                #   "LogisticRegression",
                  "RandomForest", 
                #   "SVM"
                  ]  # rápido → lento

# =========================
# Entrenamiento y evaluación
# =========================
def ejecutar_rs_y_comparar_cv_con_test(
    estimator,
    space,
    X_train,
    y_train,
    X_test,
    y_test,
    configuracion_busqueda,
    verbose=0,
):
    """
    Ejecuta RandomizedSearchCV con scoring múltiple (refit=f1_macro).
    Devuelve:
      - mejores_params: dict de hiperparámetros del mejor modelo
      - tiempo: segundos de búsqueda
      - cv: métricas promedio de validación cruzada del mejor candidato
      - test: métricas sobre el conjunto de test, incluyendo métricas adicionales
    """
    inicio = time.perf_counter()
    search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=space,
        n_iter=configuracion_busqueda["n_iter"],
        scoring=SCORING_MULTIPLE,
        refit=SCORING_REFIT,
        cv=configuracion_busqueda["cv"],
        random_state=RANDOM_STATE,
        n_jobs=configuracion_busqueda["n_jobs"],
        verbose=verbose,
        error_score=np.nan,
        return_train_score=False,
    )
    search.fit(X_train, y_train)
    fin = time.perf_counter()
    elapsed = float(fin - inicio)

    # ===================== MÉTRICAS DE CV =====================
    cv_res = search.cv_results_
    idx_best = search.best_index_

    cv_f1 = float(cv_res["mean_test_f1_macro"][idx_best])
    cv_bacc = float(cv_res["mean_test_balanced_accuracy"][idx_best])
    cv_mcc = float(cv_res["mean_test_mcc"][idx_best])
    cv_kappa = float(cv_res["mean_test_cohen_kappa"][idx_best])

    # ===================== MÉTRICAS EN TEST =====================
    best_est = search.best_estimator_
    y_pred = best_est.predict(X_test)

    # métricas ya existentes
    test_f1 = float(f1_score(y_test, y_pred, average="macro"))
    test_bacc = float(balanced_accuracy_score(y_test, y_pred))
    test_mcc = float(matthews_corrcoef(y_test, y_pred))
    test_kappa = float(cohen_kappa_score(y_test, y_pred))

    # exactitud clásica
    test_accuracy = float(accuracy_score(y_test, y_pred))

    # métricas por clase (F1 y recall)
    clases, counts = np.unique(y_test, return_counts=True)
    f1_por_clase = f1_score(y_test, y_pred, labels=clases, average=None)
    recall_por_clase = recall_score(y_test, y_pred, labels=clases, average=None)

    # recall de la clase minoritaria (la de menor soporte en y_test)
    idx_min = int(np.argmin(counts))
    recall_minoritario = float(recall_por_clase[idx_min])

    # G-mean macro: media geométrica de los recalls por clase
    if np.any(recall_por_clase < 0):
        test_gmean = float("nan")
    else:
        # si alguna clase tiene recall 0, la gmean es 0
        test_gmean = float(np.prod(recall_por_clase) ** (1.0 / len(recall_por_clase)))

    # ===================== ROC AUC y AUC-PR (si hay probabilidades) =====================
    roc_auc_macro_ovr = float("nan")
    auc_pr_macro = float("nan")
    y_proba = None

    if hasattr(best_est, "predict_proba"):
        try:
            y_proba = best_est.predict_proba(X_test)
        except Exception:
            y_proba = None

    if y_proba is not None:
        try:
            roc_auc_macro_ovr = float(
                roc_auc_score(y_test, y_proba, multi_class="ovr", average="macro")
            )
        except Exception:
            roc_auc_macro_ovr = float("nan")
        try:
            auc_pr_macro = float(
                average_precision_score(y_test, y_proba, average="macro")
            )
        except Exception:
            auc_pr_macro = float("nan")

    return dict(
        mejores_params=search.best_params_,
        tiempo=elapsed,
        cv=dict(
            f1=cv_f1,
            bacc=cv_bacc,
            mcc=cv_mcc,
            kappa=cv_kappa,
        ),
        test=dict(
            f1=test_f1,
            bacc=test_bacc,
            mcc=test_mcc,
            kappa=test_kappa,
            accuracy=test_accuracy,
            f1_por_clase=f1_por_clase,
            recall_minoritario=recall_minoritario,
            gmean=test_gmean,
            roc_auc_macro_ovr=roc_auc_macro_ovr,
            auc_pr_macro=auc_pr_macro,
            clases=clases,
        ),
    )
# =========================
# Orquestación (lista plana de tareas)
# --- Políticas globales de exclusión (en minúsculas) ---
EXCLUIR_DATASETS = {"shuttle"}  # agregar aquí otros: {"shuttle", "ecoli", ...}
# =========================

def construir_lista_plana_de_tareas(model_registry, dataset_combinations, orden_modelos,
                                    excluir_datasets=EXCLUIR_DATASETS, verbose=True):
    """
    Crea una lista plana de tareas (modelo, combinación) y aplica políticas de exclusión.
    - excluir_datasets: conjunto de nombres de dataset (en minúsculas) a excluir por completo.
    - Mantiene la política existente de omitir SVM en Shuttle aumentado si está activa.
    """
    tareas = []
    excluidos_por_dataset = 0
    excluidos_por_politica_svm_shuttle = 0

    for nombre_modelo in orden_modelos:
        for combo in dataset_combinations:
            ds = combo.dataset_logico.lower()

            # 1) Excluir datasets completos (p. ej., shuttle)
            if ds in (excluir_datasets or set()):
                excluidos_por_dataset += 1
                continue

            # 2) Política original: omitir SVM en Shuttle aumentado
            if (OMITIR_SVM_EN_SHUTTLE_AUMENTADO and
                nombre_modelo == "SVM" and
                ds == "shuttle" and
                combo.tipo_combination == "aumentado"):
                excluidos_por_politica_svm_shuttle += 1
                continue

            tareas.append((nombre_modelo, combo))

    if verbose:
        print(f"🧮 Tareas planificadas: {len(tareas)} "
              f"(excluidos por dataset: {excluidos_por_dataset}, "
              f"por política SVM-Shuttle↑: {excluidos_por_politica_svm_shuttle})")
    return tareas


# =========================
# Exportación a Excel
# =========================
def generar_archivo_excel_resultados(ruta_excel, df_resultados, df_resumen_base_vs_aumentado, df_rankings):
    """
    Escribe un archivo Excel con tres hojas:
      - 'resultados': filas con CV vs Test por (dataset, tipo, modelo, combinación).
      - 'resumen_base_vs_aumentado': promedios por dataset y modelo + delta (aum - base) en F1-Test y F1-CV.
      - 'rankings': ranking por dataset y modelo, ordenado por F1-Test.
    """
    with pd.ExcelWriter(ruta_excel, engine="xlsxwriter") as writer:
        df_resultados.to_excel(writer, sheet_name="resultados", index=False)
        df_resumen_base_vs_aumentado.to_excel(writer, sheet_name="resumen_base_vs_aumentado", index=True)
        df_rankings.to_excel(writer, sheet_name="rankings", index=False)

# =========================
# Main
# =========================

def ejecutar_experimentos_y_generar_excel():
    """
    Ejecuta todos los experimentos sobre datasets base y aumentados,
    con búsqueda aleatoria de hiperparámetros y métricas CV vs Test.
    Muestra progreso detallado por consola y exporta resultados a Excel.
    """
    # 1 -- Enumerar combinaciones base y aumentadas
    print("🔎 Enumerando combinaciones base y aumentadas...")
    combinaciones = enumerar_combinaciones_base_y_aumentadas(
        RUTA_DATASETS_BASE,
        RUTA_DATASETS_AUMENTADOS,
        RUTA_DATASETS_CLASICOS,
    )


    if not combinaciones:
        print("❌ No se encontraron combinaciones de datasets.")
        return

    datasets_con_base = {c.dataset_logico for c in combinaciones if c.tipo_combination == "base"}
    if not datasets_con_base:
        print("❌ No hay datasets base para comparar.")
        return

    # 2 -- Construir plan de tareas
    """
    Con las combinaciones descubiertas, construimos plan de tareas
    """
    tareas = construir_lista_plana_de_tareas(
        REGISTRO_MODELOS,
        combinaciones,
        ORDEN_MODELOS,
        excluir_datasets={"shuttle"},   # datasets a excluir por completo
        verbose=True
    )
    total_tareas = len(tareas)
    print(f"📦 Total de tareas planificadas: {total_tareas}")

    registros = []
    inicio_total = time.perf_counter()

    # 3 -- Ejecutar tareas
    for idx, (nombre_modelo, combo) in enumerate(tareas, start=1):
        print(f"\n{'='*80}")
        print(f"🏁 [{idx}/{total_tareas}] Dataset: {combo.dataset_logico} | "
              f"Tipo: {combo.tipo_combination} | Modelo: {nombre_modelo}")
        print(f"📂 Train: {os.path.basename(combo.ruta_train_csv)}")

        try:
            X_train, y_train = cargar_matriz_caracteristicas_y_etiquetas_desde_csv(combo.ruta_train_csv)
            X_test, y_test = cargar_matriz_caracteristicas_y_etiquetas_desde_csv(combo.ruta_test_csv)
        except Exception as e:
            print(f"❌ Error leyendo archivos CSV: {e}")
            continue

        configuracion_busqueda = definir_configuracion_busqueda_para_dataset(
            X_train, combo.dataset_logico, combo.tipo_combination
        )
        print(f"⚙️  Configuración de búsqueda: "
              f"n_iter={configuracion_busqueda['n_iter']}, "
              f"folds={configuracion_busqueda['cv'].n_splits}, "
              f"n_jobs={configuracion_busqueda['n_jobs']}")

        # 3.1 -- Construir estimador y espacio de hiperparámetros
        # por modelo seleccionado obtener el estimador y espacio
        estimator, space = REGISTRO_MODELOS[nombre_modelo]()
        print(f"🚀 Iniciando RandomizedSearchCV...")

        try:
            resultados = ejecutar_rs_y_comparar_cv_con_test(
                estimator, space, X_train, y_train, X_test, y_test,
                configuracion_busqueda=configuracion_busqueda, verbose=1
            )
        except Exception as e:
            print(f"❌ Error durante la búsqueda: {e}")
            continue

        print(f"✅ Búsqueda completada en {resultados['tiempo']:.2f} s")
        print(f"📊 F1(CV): {resultados['cv']['f1']:.4f} | "
              f"F1(Test): {resultados['test']['f1']:.4f}")

        # formateo F1 por clase como string legible para Excel
        f1_por_clase = resultados["test"]["f1_por_clase"]
        f1_por_clase_str = ";".join([f"{v:.4f}" for v in f1_por_clase])

        registros.append(asdict(RegistroRendimiento(
            dataset_logico=combo.dataset_logico,
            tipo_combination=combo.tipo_combination,
            nombre_modelo_aprendizaje=nombre_modelo,
            tecnica_aumento=combo.tecnica_aumento,
            valor_densidad=combo.valor_densidad,
            valor_riesgo=combo.valor_riesgo,
            criterio_pureza=combo.criterio_pureza,
            grado_limpieza=combo.grado_limpieza,
            cantidad_train=int(X_train.shape[0]),
            cantidad_test=int(X_test.shape[0]),
            cantidad_caracteristicas=int(X_train.shape[1]),
            cv_f1_macro=resultados["cv"]["f1"],
            cv_balanced_accuracy=resultados["cv"]["bacc"],
            cv_mcc=resultados["cv"]["mcc"],
            cv_cohen_kappa=resultados["cv"]["kappa"],
            test_f1_macro=resultados["test"]["f1"],
            test_balanced_accuracy=resultados["test"]["bacc"],
            test_mcc=resultados["test"]["mcc"],
            test_cohen_kappa=resultados["test"]["kappa"],
            test_accuracy=resultados["test"]["accuracy"],
            test_f1_por_clase=f1_por_clase_str,
            test_recall_minoritario=resultados["test"]["recall_minoritario"],
            test_gmean=resultados["test"]["gmean"],
            test_roc_auc_macro_ovr=resultados["test"]["roc_auc_macro_ovr"],
            test_auc_pr_macro=resultados["test"]["auc_pr_macro"],
            mejores_hiperparametros=str(resultados["mejores_params"]),
            tiempo_busqueda_seg=float(resultados["tiempo"]),
        )))


        gc.collect()

    # ----------------- DataFrames finales -----------------
    print("\n📊 Compilando resultados globales...")
    df_resultados = pd.DataFrame(registros)

    resumen = (df_resultados
        .groupby(["dataset_logico", "nombre_modelo_aprendizaje", "tipo_combination"])
        .agg(cv_f1_prom=("cv_f1_macro", "mean"),
             test_f1_prom=("test_f1_macro", "mean"))
        .unstack("tipo_combination"))
    for col in [("cv_f1_prom", "base"), ("cv_f1_prom", "aumentado"), ("test_f1_prom", "base"), ("test_f1_prom", "aumentado")]:
        if col not in resumen.columns:
            resumen[col] = np.nan
    resumen = resumen.assign(
        delta_cv_f1 = resumen[("cv_f1_prom", "aumentado")] - resumen[("cv_f1_prom", "base")],
        delta_test_f1 = resumen[("test_f1_prom", "aumentado")] - resumen[("test_f1_prom", "base")]
    )

    lista_rankings = []
    for ds in sorted(df_resultados["dataset_logico"].unique()):
        sub_ds = df_resultados[df_resultados["dataset_logico"] == ds]
        for mdl in ORDEN_MODELOS:
            sub_mdl = sub_ds[sub_ds["nombre_modelo_aprendizaje"] == mdl].copy()
            if sub_mdl.empty:
                continue
            sub_mdl.sort_values("test_f1_macro", ascending=False, inplace=True)
            sub_mdl["ranking"] = range(1, len(sub_mdl) + 1)
            lista_rankings.append(sub_mdl[[
                "dataset_logico","nombre_modelo_aprendizaje","ranking",
                "valor_riesgo","valor_densidad","criterio_pureza",
                "tipo_combination","test_f1_macro","cv_f1_macro",
                "tecnica_aumento","mejores_hiperparametros"
            ]])
    df_rankings = pd.concat(lista_rankings, ignore_index=True) if lista_rankings else pd.DataFrame()

    generar_archivo_excel_resultados(
        ruta_excel=NOMBRE_ARCHIVO_EXCEL,
        df_resultados=df_resultados,
        df_resumen_base_vs_aumentado=resumen,
        df_rankings=df_rankings
    )

    fin_total = time.perf_counter()
    duracion = round(fin_total - inicio_total, 2)
    print(f"\n🏁 Ejecución total completada en {duracion} s")
    print(f"📘 Archivo Excel generado: {NOMBRE_ARCHIVO_EXCEL}")


ejecutar_experimentos_y_generar_excel()


🔎 Enumerando combinaciones base y aumentadas...
📂 Explorando carpeta base: ../datasets/datasets_aumentados/base/
  ⚪ Omitido (no es *_train.csv): ecoli_test.csv
  ⚪ Omitido (no es *_train.csv): glass_test.csv
  ⚪ Omitido (no es *_train.csv): wdbc_test.csv
📂 Explorando carpeta clásicos: ../datasets/datasets_aumentados/resampler_clasicos/
  ✅ Clásico: adasyn_wdbc_train.csv -> test base: wdbc_test.csv
  ✅ Clásico: borderlinesmote_glass_train.csv -> test base: glass_test.csv
  ✅ Clásico: borderlinesmote_wdbc_train.csv -> test base: wdbc_test.csv
  ✅ Clásico: smote_glass_train.csv -> test base: glass_test.csv
  ✅ Clásico: smote_wdbc_train.csv -> test base: wdbc_test.csv
📂 Explorando carpeta aumentados: ../datasets/datasets_aumentados/
  ⚪ Omitido (no es *_train.csv): .gitignore
  ⚪ Omitido (no es *_train.csv): base
  ⚪ Omitido (no es *_train.csv): logs
  ✅ Aumentado: pcsmote_ecoli_D25_R25_Pentropia_I0_train.csv -> test base: ecoli_test.csv
  ✅ Aumentado: pcsmote_ecoli_D25_R25_Pproporcion_I0

c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=40. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


✅ Búsqueda completada en 6.42 s
📊 F1(CV): 0.7007 | F1(Test): 0.8846

🏁 [2/62] Dataset: glass | Tipo: base | Modelo: RandomForest
📂 Train: glass_train.csv
⚙️  Configuración de búsqueda: n_iter=40, folds=5, n_jobs=2
🚀 Iniciando RandomizedSearchCV...
Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=40. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


✅ Búsqueda completada en 1.91 s
📊 F1(CV): 0.6710 | F1(Test): 0.8374

🏁 [3/62] Dataset: wdbc | Tipo: base | Modelo: RandomForest
📂 Train: wdbc_train.csv
⚙️  Configuración de búsqueda: n_iter=40, folds=5, n_jobs=2
🚀 Iniciando RandomizedSearchCV...
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

✅ Búsqueda completada en 2.79 s
📊 F1(CV): 0.9596 | F1(Test): 0.9905

🏁 [61/62] Dataset: wdbc | Tipo: aumentado | Modelo: RandomForest
📂 Train: pcsmote_wdbc_D75_R75_Pentropia_I0_train.csv
⚙️  Configuración de búsqueda: n_iter=40, folds=5, n_jobs=2
🚀 Iniciando RandomizedSearchCV...
Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=40. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


✅ Búsqueda completada en 2.66 s
📊 F1(CV): 0.9544 | F1(Test): 0.9810

🏁 [62/62] Dataset: wdbc | Tipo: aumentado | Modelo: RandomForest
📂 Train: pcsmote_wdbc_D75_R75_Pproporcion_I0_train.csv
⚙️  Configuración de búsqueda: n_iter=40, folds=5, n_jobs=2
🚀 Iniciando RandomizedSearchCV...
Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\FamiliaNatelloMedina\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=40. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


✅ Búsqueda completada en 2.81 s
📊 F1(CV): 0.9596 | F1(Test): 0.9905

📊 Compilando resultados globales...

🏁 Ejecución total completada en 156.48 s
📘 Archivo Excel generado: ../resultados\resultados_RS_cv_vs_test.xlsx


In [17]:
# from pathlib import Path
# import pandas as pd

# # Directorio base = carpeta del notebook (CWD en Jupyter)
# BASE = Path.cwd()

# # Carpeta resultados: hermana de "notebooks"
# CARPETA_RESULTADOS = (BASE / "../resultados").resolve()

# # Archivos
# ARCH_SIN = CARPETA_RESULTADOS / "resultados_RS_cv_vs_test_sin_isolation.xlsx"
# ARCH_CON = CARPETA_RESULTADOS / "resultados_RS_cv_vs_test_con_isolation.xlsx"

# # Comprobaciones útiles
# print("CWD:", BASE)
# print("Resultados:", CARPETA_RESULTADOS)
# print("Existe SIN:", ARCH_SIN.exists(), ARCH_SIN)
# print("Existe CON:", ARCH_CON.exists(), ARCH_CON)

# # Carga
# sin_iso = pd.read_excel(ARCH_SIN)
# con_iso = pd.read_excel(ARCH_CON)


In [18]:
# from graficador_resultados import GraficadorResultados

# graficador = GraficadorResultados(sin_iso, con_iso)
# graficador.preparar_datos()

# # Un bloque para 'glass'
# graficador.panel_por_dataset(dataset="glass",
#                              modelo=None,
#                              metrica_heatmap="test_f1_macro",
#                              usar_delta_heatmap=False,
#                              figsize=(14,12),
#                              guardar_path="figs/panel_glass.png")
